In [25]:
import cv2

In [26]:
#Coloque o caminho em que está a pasta 'cg-proc' que foi clonada do repositório
master_path = "/home/ken/KR/USP/7S/CG/"

In [27]:
# Paths das imagens
one_each = "{}cg-proc/img/one_each.jpeg".format(master_path)
glued = "{}cg-proc/img/glued.jpeg".format(master_path)
mess = "{}cg-proc/img/mess.jpeg".format(master_path)
real_and_twentyFive = "{}cg-proc/img/real_and_twentyFive.jpeg".format(master_path)

In [28]:
# Dict para usar o nome da variável na hora de salvar o arquivo alterado.
images_dict = {
    'one_each': one_each,
    'glued': glued,
    'mess': mess,
    'real_and_twentyFive': real_and_twentyFive
}


In [29]:
#Funções de auxilio para plicar as transformações na imagem

def rgb_to_gray(image):
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

def threshold(image, threshold_value):
    
    _, binarized_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return binarized_image

def negative(image):
    
    negative_image = cv2.bitwise_not(image)
    return negative_image

In [30]:
# Primeira etapa: carregando imagens e fazendo um primeiro threshold
for name, img_path in images_dict.items():
    image = cv2.imread(img_path)
    gray_image = rgb_to_gray(image)
    threshold_value = 135
    binarized_image = threshold(gray_image, threshold_value)
    cv2.imwrite("{}cg-proc/img/edited/t_hold_1_{}.jpeg".format(master_path, name), binarized_image)

In [31]:
# Segunda etapa: um novo threshold para deixar a imagem mais "flat": sem as sombras das imagens
for name, img_path in images_dict.items():
    image = cv2.imread(img_path)
    gray_image = rgb_to_gray(image)
    threshold_value = 110
    binarized_image = threshold(gray_image, threshold_value)
    cv2.imwrite("{}cg-proc/img/edited/t_hold_2_{}.jpeg".format(master_path, name), binarized_image)

In [32]:
# Terceira etapa: fazendo o negativo da imagem para trabalhar com as cores certas.
images_dict_edited = {
    'one_each_': "{}cg-proc/img/edited/t_hold_2_one_each.jpeg".format(master_path),
    'glued_': "{}cg-proc/img/edited/t_hold_2_glued.jpeg".format(master_path),
    'mess_': "{}cg-proc/img/edited/t_hold_2_mess.jpeg".format(master_path),
    'real_and_twentyFive_': "{}cg-proc/img/edited/t_hold_2_real_and_twentyFive.jpeg".format(master_path)
}

for name, img_path in images_dict_edited.items():
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    threshold_value = 135
    binarized_image = threshold(image, threshold_value)
    negative_image = negative(binarized_image)
    cv2.imwrite("{}cg-proc/img/edited/{}_negative.jpeg".format(master_path, name), negative_image)